In [5]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    "./data",
    glob="*.md",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

documents = loader.load()
print(f"✅ Loaded {len(documents)} documents (files).")
for d in documents:
    print(" -", d.metadata.get("source"))



✅ Loaded 4 documents (files).
 - data\bio.md
 - data\project.md
 - data\qa.md
 - data\skills.md


In [6]:
import re
from copy import deepcopy

def infer_doc_type(source_path: str) -> str:
    name = os.path.basename(source_path).lower()
    if "bio" in name:
        return "bio"
    if "project" in name:
        return "projects"
    if "skill" in name:
        return "skills"
    if name in ("qa.md", "qa_seed.md") or "qa" in name:
        return "qa"
    return "general"

def split_markdown_by_headings(text: str):
    """
    Split markdown text into sections using ## / ### headings.
    Keeps heading lines within each chunk.
    """
    lines = text.splitlines()
    sections = []
    current = []
    for line in lines:
        if re.match(r"^#{2,3}\s+", line.strip()):  # ## or ###
            if current:
                sections.append("\n".join(current).strip())
                current = []
        current.append(line)
    if current:
        sections.append("\n".join(current).strip())

    # Fallback if no headings found: keep whole doc as one section
    return [s for s in sections if s] or [text.strip()]

def chunk_long_section(section_text: str, max_chars=2200, overlap=200):
    """
    If a heading-section is too large, sub-split it safely by characters.
    (We do this only when needed.)
    """
    if len(section_text) <= max_chars:
        return [section_text]

    chunks = []
    start = 0
    while start < len(section_text):
        end = min(start + max_chars, len(section_text))
        chunk = section_text[start:end]
        chunks.append(chunk.strip())
        if end == len(section_text):
            break
        start = max(0, end - overlap)
    return chunks

# Build chunks with metadata
chunks = []
for doc in documents:
    src = doc.metadata.get("source", "unknown")
    doc_type = infer_doc_type(src)

    # Split doc into heading-sections
    sections = split_markdown_by_headings(doc.page_content)

    for sec in sections:
        # Extract the first heading line as section title (if exists)
        m = re.search(r"^(#{2,3})\s+(.+)$", sec, flags=re.MULTILINE)
        section_title = m.group(2).strip() if m else "NoHeading"

        # Sub-split only if section is too big
        for sub in chunk_long_section(sec, max_chars=2200, overlap=200):
            d2 = deepcopy(doc)
            d2.page_content = sub

            # Add clean metadata (for citations/routing later)
            d2.metadata = dict(d2.metadata) if d2.metadata else {}
            d2.metadata["source_file"] = os.path.basename(src)
            d2.metadata["doc_type"] = doc_type
            d2.metadata["section"] = section_title

            # Optional: project name extraction for project catalog style
            # If section starts with "## Project Name"
            if doc_type == "projects":
                pm = re.match(r"^##\s+(.+)$", sub.strip().splitlines()[0].strip())
                if pm:
                    d2.metadata["project_name"] = pm.group(1).strip()

            chunks.append(d2)

print(f"✅ Heading-based split produced {len(chunks)} chunks.")
print("\n--- Sample chunk ---")
print(chunks[0].metadata)
print(chunks[0].page_content[:800])
print("-------------------")


✅ Heading-based split produced 117 chunks.

--- Sample chunk ---
{'source': 'data\\bio.md', 'source_file': 'bio.md', 'doc_type': 'bio', 'section': 'NoHeading'}
# Ankit Sharma — AI/ML Portfolio Profile
-------------------


In [9]:
import os
import hashlib
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_community.vectorstores.upstash import UpstashVectorStore

# 1. Load environment variables
load_dotenv()

# 2. Load the embedding model via API
print("⏳ Loading embedding model via Hugging Face API...")
embedding_model = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-MiniLM-L6-v2",
    task="feature-extraction",
    huggingfacehub_api_token=os.environ.get("HF_TOKEN")
)

# 3. Connect to Upstash Vector 
print("⏳ Connecting to Upstash Vector...")
vector_store = UpstashVectorStore(
    embedding=embedding_model
)

# 4. Generate unique, repeatable IDs for each chunk based on its text
chunk_ids = []
# Make sure your 'chunks' variable from the split cell above is still loaded!
for chunk in chunks:
    chunk_hash = hashlib.md5(chunk.page_content.encode('utf-8')).hexdigest()
    chunk_ids.append(chunk_hash)

# 5. Upload with the specific IDs
print(f"⏳ Uploading {len(chunks)} chunks to the cloud...")
vector_store.add_documents(documents=chunks, ids=chunk_ids)

print(f"✅ Success! Uploaded {len(chunks)} chunks to Upstash Vector!")

⏳ Loading embedding model via Hugging Face API...
⏳ Connecting to Upstash Vector...
⏳ Uploading 117 chunks to the cloud...
✅ Success! Uploaded 117 chunks to Upstash Vector!


## not required 


from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)

query = "Tell me about the Brain Tumor Detection project"
results = vector_store.similarity_search(query, k=6)

print(f"\n🔍 Query: {query}")
print("-" * 60)
for i, doc in enumerate(results, 1):
    print(f"📄 Result {i}")
    print("  source_file:", doc.metadata.get("source_file"))
    print("  doc_type:", doc.metadata.get("doc_type"))
    print("  section:", doc.metadata.get("section"))
    print("  project_name:", doc.metadata.get("project_name"))
    print("  preview:", doc.page_content[:220].replace("\n", " ") + " ...")
    print("-" * 60)


import os
from getpass import getpass
from huggingface_hub import InferenceClient
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Token
if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = getpass("🔑 Enter your Hugging Face Token: ")

# Retriever
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 25, "lambda_mult": 0.6}
)


def format_docs(docs):
    # Add source labels to help grounding + future citations
    lines = []
    for d in docs:
        src = d.metadata.get("source_file", "unknown")
        sec = d.metadata.get("section", "unknown_section")
        lines.append(f"[Source: {src} | Section: {sec}]\n{d.page_content}")
    return "\n\n".join(lines)

def query_zephyr_chat(prompt_text):
    client = InferenceClient(api_key=os.environ["HF_TOKEN"])
    messages = [{"role": "user", "content": prompt_text}]
    resp = client.chat_completion(
        model="HuggingFaceH4/zephyr-7b-beta",
        messages=messages,
        max_tokens=200,
        temperature=0.1
    )
    return resp.choices[0].message.content

# 5) Prompt (still “about you”, but allows brief general explanations)
template = """You are Ankit's AI Portfolio Assistant.

Use ONLY the Context for Ankit-specific facts. If not in Context, say: "I don't have that info about Ankit."
Answer in 3–6 short bullet points using "-" only. No numbering, no paragraphs, no extra sections.

If query matches hi|hello|thanks|thank you|ok|bye, return a short canned response (no RAG call).
“Hi” → “Hi! Ask me about projects, skills, internship, or achievements.”

“Thanks” → “You’re welcome—want a project summary or resume bullets?”

Context:
{context}

Question: {question}
Answer:
"""


prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | (lambda x: x.to_string())
    | query_zephyr_chat
)

print("✅ Chain ready.")
print("=" * 60)
print(rag_chain.invoke("what is ankit's cgpa in college?"))
print("=" * 60)
